Python的标准库提供了两个模块：_thread和threading，_thread是低级模块，threading是高级模块，对_thread进行了封装。  
英文文档  
https://docs.python.org/3.6/library/threading.html  
中文材料   
https://blog.csdn.net/l835311324/article/details/86608850  
https://www.cnblogs.com/ygbh/p/11990315.html  
https://www.cnblogs.com/enet01/p/6027368.html  
https://www.cnblogs.com/tkqasn/p/5700281.html  
https://www.dazhuanlan.com/2019/11/28/5ddf738b5e5bc/   
https://blog.csdn.net/brucewong0516  
https://blog.csdn.net/brucewong0516/article/details/84025027  
https://blog.csdn.net/brucewong0516/article/details/81028716  
https://blog.csdn.net/brucewong0516/article/details/81050939  
https://blog.csdn.net/brucewong0516/article/details/84587522  
https://blog.csdn.net/brucewong0516/article/details/84588804  
https://blog.csdn.net/brucewong0516/article/details/84589616  
https://blog.csdn.net/brucewong0516/article/details/84589806  
https://www.cnblogs.com/yoyoketang/p/8337118.html  

In [5]:
import time, threading

In [19]:
con1 = threading.Condition()

In [20]:
"""获取锁"""
con1.acquire()

True

In [4]:
"""通知所有正在等待的线程，notify和nofify_all不会释放锁，并且需要在release（）之前调用"""
con1.notify_all() 

In [5]:
"""通知正在等待的某个线程，默认是第一个等待的线程"""
con1.notify() 

In [ ]:
"""释放锁"""
con1.release()

In [ ]:
"""将当前线程处于等待状态，并释放锁。
可以被其他线程使用notify和notify_all函数唤醒，被唤醒后会继续等待上锁，上锁后继续执行下面代码.
"""
con1.wait() 

In [7]:
"""注意1：wait()必须在已获得Lock前提下才能调用，也就是con1.acquire()。wait()会主动释放Lock。
注意2：最后用了两个con1.release()，这是因为前面锁了两次，第一次是wait被f2的con1.notify()通知时自动取得锁，第二次是又在wait()后多写了一个con1.acquire()，这里就是为了提醒重复锁定问题。
注意3：大家都是对con1这同一个变量进行操作，如果各自分开定义con1,con2则互相不能操作。
注意4：可以认为，Condition除了Lock带有的锁定池外，还包含一个等待池，池中的线程处于状态图中的等待阻塞状态，直到另一个线程调用notify()/notifyAll()通知；得到通知后线程进入锁定池等待锁定。
注意5：notify()不会主动释放Lock。所以最后要用con1.release()释放。
"""
def f1():
    con1.acquire()
    con1.notify()
    print('f1b')
    con1.wait()
    print('f1c')
    con1.acquire()
    con1.notify()
    con1.release()
    con1.release()

In [8]:
def f2():
    con1.acquire()
    con1.notify()
    print('f2b')
    con1.wait()
    print('f2c')
    con1.acquire()
    con1.notify()
    con1.release()

In [9]:
t1 = threading.Thread(target=f1)
t2 = threading.Thread(target=f2)

In [10]:
t1.start()

f1b


In [11]:
t2.start()

f2b
f1c
f2c


____

In [12]:
threading.enumerate()

[<_MainThread(MainThread, started 22412)>,
 <Thread(Thread-4, started daemon 24508)>,
 <Heartbeat(Thread-5, started daemon 2892)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 23832)>,
 <ParentPollerWindows(Thread-3, started daemon 24824)>]

In [5]:
threading.active_count()

5

In [6]:
threading.current_thread()

<_MainThread(MainThread, started 14052)>

In [7]:
threading.get_ident()

14052

In [8]:
threading.main_thread()

<_MainThread(MainThread, started 14052)>

In [9]:
t = threading.Thread()

In [10]:
t.setName('wanghl')

In [11]:
t.getName()

'wanghl'

In [12]:
t.name

'wanghl'

In [15]:
t.ident

In [14]:
t.daemon

False

In [56]:
"""新线程执行的代码:"""
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.


In [57]:
"""假定这是你的银行存款:"""

balance = 0

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n

def run_thread(n):
    for i in range(2000000):
        change_it(n)

t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

26


In [58]:
balance = 0
lock = threading.Lock()

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(100000):
        # 先要获取锁:
        lock.acquire()
        try:
            # 放心地改吧:
            change_it(n)
        finally:
            # 改完了一定要释放锁:
            lock.release()
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


In [60]:
"""每个函数一层一层调用都这么传参数那还得了？用全局变量？也不行，因为每个线程处理不同的Student对象，不能共享。
"""
def process_student(name):
    std = Student(name)
    # std是局部变量，但是每个函数都要用它，因此必须传进去：
    do_task_1(std)
    do_task_2(std)

def do_task_1(std):
    do_subtask_1(std)
    do_subtask_2(std)

def do_task_2(std):
    do_subtask_2(std)
    do_subtask_2(std)

In [59]:
global_dict = {}

def std_thread(name):
    std = Student(name)
    # 把std放到全局变量global_dict中：
    global_dict[threading.current_thread()] = std
    do_task_1()
    do_task_2()

def do_task_1():
    # 不传入std，而是根据当前线程查找：
    std = global_dict[threading.current_thread()]
    ...

def do_task_2():
    # 任何函数都可以查找出当前线程的std变量：
    std = global_dict[threading.current_thread()]
    ...

In [50]:
"""ThreadLocal最常用的地方就是为每个线程绑定一个数据库连接，HTTP请求，用户身份信息等，这样一个线程的所有调用到的处理函数都可以非常方便地访问这些资源。
"""
# 创建全局ThreadLocal对象:
local_school = threading.local()

def process_student():
    # 获取当前线程关联的student:
    std = local_school.student
    print('Hello, %s (in %s)' % (std, threading.current_thread().name))

def process_thread(name):
    # 绑定ThreadLocal的student:
    local_school.student = name
    process_student()

t1 = threading.Thread(target= process_thread, args=('Alice',), name='Thread-A')
t2 = threading.Thread(target= process_thread, args=('Bob',), name='Thread-B')
t1.start()
t2.start()
t1.join()
t2.join()

Hello, Alice (in Thread-A)
Hello, Bob (in Thread-B)


In [63]:
"""条件变量,用于复杂的线程间同步锁

需求:
    男:小姐姐,你好呀!
    女:哼,想泡老娘不成?
    男:对呀,想泡你
    女:滚蛋,门都没有!
    男:切,长这么丑, 还这么吊...
    女:关你鸟事!

"""
class Boy(threading.Thread):
    def __init__(self, name, condition):
        super().__init__(name=name)
        self.condition = condition

    def run(self):
        with self.condition:
            print("{}:小姐姐,你好呀!".format(self.name))
            time.sleep(0.1)
            self.condition.wait()
            self.condition.notify()

            print("{}:对呀,想泡你".format(self.name))
            time.sleep(0.1)
            self.condition.wait()
            self.condition.notify()

            print("{}:切,长这么丑, 还这么吊...".format(self.name))
            time.sleep(0.1)
            self.condition.wait()
            self.condition.notify()


class Girl(threading.Thread):
    def __init__(self, name, condition):
        super().__init__(name=name)
        self.condition = condition

    def run(self):
        with self.condition:
            print("{}:哼,想泡老娘不成?".format(self.name))
            time.sleep(1)
            self.condition.notify()
            self.condition.wait()

            print("{}:滚蛋,门都没有!".format(self.name))
            time.sleep(1)
            self.condition.notify()
            self.condition.wait()

            print("{}:关你鸟事!".format(self.name))
            time.sleep(1)
            self.condition.notify()
            self.condition.wait()
condition = threading.Condition()

boy_thread = Boy('男', condition)
girl_thread = Girl('女', condition)

boy_thread.start()

girl_thread.start()

男:小姐姐,你好呀!
女:哼,想泡老娘不成?
男:对呀,想泡你
女:滚蛋,门都没有!
男:切,长这么丑, 还这么吊...
女:关你鸟事!


In [27]:
"""示例1：创建Thread实例，传递函数"""
def read():
    for x in range(3):
        print('在%s,正在读书' % time.ctime())
        time.sleep(1)

def write():
    for x in range(3):
        print('在%s,正在写字' % time.ctime())
        time.sleep(1)

def main():

    read_threads = []  # 用来存放执行read函数线程的列表
    write_threads = []  # 用来存放执行write函数线程的列表

    for i in range(1,2):  # 创建1个线程用于read()，并添加到read_threads列表
        t = threading.Thread(target=read) # 执行的函数如果需要传递参数，threading.Thread(target=函数名,args=(参数，逗号隔开))
        read_threads.append(t)

    for i in range(1,2): # 创建1个线程执行write()，并添加到write_threads列表
        t = threading.Thread(target=write) # 执行的函数如果需要传递参数，threading.Thread(target=函数名,args=(参数，逗号隔开))
        write_threads.append(t)

    for i in range(0,1):  # 启动存放在read_threads和write_threads列表中的线程
        read_threads[i].start()
        write_threads[i].start()

if __name__ == '__main__':
    main()

在Tue Nov 17 10:35:06 2020,正在读书
在Tue Nov 17 10:35:06 2020,正在写字
在Tue Nov 17 10:35:07 2020,正在读书
在Tue Nov 17 10:35:07 2020,正在写字
在Tue Nov 17 10:35:08 2020,正在读书
在Tue Nov 17 10:35:08 2020,正在写字


In [26]:
"""示例2：为了让线程更好的封装，可以用threading模块下的Thread类，继承这个类，然后实现run方法，线程就回自动运行run方法中的代码"""
class read_thread(threading.Thread):
    def run(self):
        for x in range(3):
            print('在%s,正在读书(当前线程：%s)' % (time.ctime(),threading.current_thread()))
            time.sleep(1)

class write_thread(threading.Thread):
    def run(self):
        for x in range(3):
            print('在%s,正在写字(当前线程：%s)' % (time.ctime(),threading.current_thread()))
            time.sleep(1)

def main():

    read_threads = []  # 用来存放执行read函数线程的列表
    write_threads = []  # 用来存放执行write函数线程的列表

    for i in range(1,2):  # 创建1个线程用于read()，并添加到read_threads列表
        t = read_thread()  # 创建实例存放
        read_threads.append(t)

    for i in range(1,2): # 创建1个线程执行write()，并添加到write_threads列表
        t = write_thread()
        write_threads.append(t)

    for i in range(0,1):  # 启动存放在read_threads和write_threads列表中的线程
        read_threads[i].start()
        write_threads[i].start()

if __name__ == '__main__':
    main()

在Tue Nov 17 10:34:23 2020,正在读书(当前线程：<read_thread(Thread-90, started 23288)>)
在Tue Nov 17 10:34:23 2020,正在写字(当前线程：<write_thread(Thread-91, started 21080)>)
在Tue Nov 17 10:34:24 2020,正在读书(当前线程：<read_thread(Thread-90, started 23288)>)
在Tue Nov 17 10:34:24 2020,正在写字(当前线程：<write_thread(Thread-91, started 21080)>)
在Tue Nov 17 10:34:25 2020,正在读书(当前线程：<read_thread(Thread-90, started 23288)>)
在Tue Nov 17 10:34:25 2020,正在写字(当前线程：<write_thread(Thread-91, started 21080)>)


In [31]:
"""这个main返回结果显然是不对的，这就是因为全局变量是共享的，线程运行又是无序的，而且value+1的命令次数变多，就会2个线程可能同时执行+1这时候数据就很容易发生混乱。
"""
value = 0
def add_value():
    global value
    for x in range(10000000):
        value += 1
    print('value:',value)

def main():
    threads = []
    for i in range(0,4):
        t = threading.Thread(target=add_value)
        threads.append(t)

    for i in range(0,4):
        threads[i].start()

if __name__ == '__main__':
    main()

value: 10235429
value: 14422522
value: 17547708
value: 18186105


In [30]:
"""加锁后就正确了。
锁有两种状态：锁定和未锁定。而且也只有两个函数;获取锁和释放锁。
当多线程争夺锁的时候，允许第一个获得锁的线程进入临界区，并执行代码，所有之后叨叨的线程都被阻塞，当第一个线程执行结束，退出临界区，释放锁，此时，其他等待的线程可以获得锁进入临界区。
不过要记住，被阻塞的线程是无序的。
"""
value = 0
lock = threading.Lock() # 创建锁示例

def add_value():
    global value
    lock.acquire() # 获得锁
    for x in range(10000000):
        value += 1
    lock.release() # 释放锁
    print('value:',value)

def main():
    threads = []
    for i in range(0,4):
        t = threading.Thread(target=add_value)
        threads.append(t)

    for i in range(0,4):
        threads[i].start()

if __name__ == '__main__':
    main()

value: 10000000
value: 20000000
value: 30000000
value: 40000000


In [1]:
"""LOCK锁机制存在一个问题，上锁是一个很耗费CPU资源的行为，这时候就可以考虑使用Condition对象,
threading.Condition()可以再没有数据的时候处于阻塞等待状态，一旦有合适的数据，还可以使用notify相关的函数来通知其他处于等待状态的线程，
这样就可以不用做一些无用的上锁和解锁操作，可以提高程序的性能
"""

import  time
import  threading
import random

In [19]:
con1 = threading.Condition()

In [20]:
"""获取锁"""
con1.acquire()

True

In [4]:
"""通知所有正在等待的线程，notify和nofify_all不会释放锁，并且需要在release（）之前调用"""
con1.notify_all() 

In [5]:
"""通知正在等待的某个线程，默认是第一个等待的线程"""
con1.notify() 

In [ ]:
"""释放锁"""
con1.release()

In [ ]:
"""将当前线程处于等待状态，并释放锁。
可以被其他线程使用notify和notify_all函数唤醒，被唤醒后会继续等待上锁，上锁后继续执行下面代码.
"""
con1.wait() 

In [33]:
"""创建两个类，Producer用于赚钱，Consumer用于消费。
Consumer每次花钱的时候会看一下金库的总额够不够，如果不够就会调用gCondition.wait()将线程处于等待，
当Producer赚钱了，就会调用gCondition.notify_all()，提醒等待的线程，当线程被唤醒之后会继续等待上锁，上锁后继续执行下面代码
"""
gMoney = 1000
gCondition = threading.Condition()
gTotalTimes = 10
gTimes = 0


class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:
            money = random.randint(100,1000)
            gCondition.acquire()
            if gTimes >= gTotalTimes:
                gCondition.release()
                break
            gMoney += money
            print("%s生产者生产了%d元，总额%d元" % (threading.current_thread(), money, gMoney))
            gTimes +=1
            gCondition.notify_all()
            gCondition.release()
            time.sleep(0.5)

class Consumer(threading.Thread):
    def run(self):
        global gMoney
        while True:
            price = random.randint(500,1000)
            gCondition.acquire()
            while gMoney < price:
                if gTimes>= gTotalTimes:
                    gCondition.release()
                    return
                print('%s消费者准备消费%d,剩余%d,不足！' % (threading.current_thread(), price, gMoney))
                gCondition.wait()
            gMoney -= price
            print('%s消费者消费了%d元，剩余%d' % (threading.current_thread(),price,gMoney))
            gCondition.release()
            time.sleep(0.5)

def main():
    for x in range(3):
        t = Consumer(name='消费者线程%d'%x)
        t.start()
    for x in range(5):
        t = Producer(name = '生产者线程%d'%x )
        t.start()

if __name__ == '__main__':
    main()

<Consumer(消费者线程0, started 18476)>消费者消费了555元，剩余445
<Consumer(消费者线程1, started 24016)>消费者准备消费794,剩余445,不足！
<Consumer(消费者线程2, started 13876)>消费者准备消费945,剩余445,不足！
<Producer(生产者线程0, started 12892)>生产者生产了120元，总额565元
<Consumer(消费者线程2, started 13876)>消费者准备消费945,剩余565,不足！
<Consumer(消费者线程1, started 24016)>消费者准备消费794,剩余565,不足！
<Producer(生产者线程1, started 20972)>生产者生产了528元，总额1093元
<Consumer(消费者线程2, started 13876)>消费者消费了945元，剩余148
<Consumer(消费者线程1, started 24016)>消费者准备消费794,剩余148,不足！
<Producer(生产者线程2, started 20828)>生产者生产了230元，总额378元
<Consumer(消费者线程1, started 24016)>消费者准备消费794,剩余378,不足！
<Producer(生产者线程3, started 22644)>生产者生产了992元，总额1370元
<Consumer(消费者线程1, started 24016)>消费者消费了794元，剩余576
<Producer(生产者线程4, started 15788)>生产者生产了265元，总额841元
<Consumer(消费者线程0, started 18476)>消费者消费了733元，剩余108
<Producer(生产者线程0, started 12892)>生产者生产了782元，总额890元
<Consumer(消费者线程2, started 13876)>消费者消费了604元，剩余286
<Producer(生产者线程1, started 20972)>生产者生产了124元，总额410元
<Producer(生产者线程2, started 20828)>生产者生产了231元，总额641元
<Consumer(消费者线程1

In [12]:
"""单线程执行"""
def hello_for():
    print("张三：你好 世界")
    time.sleep(1)
    print("张三：hi")
    time.sleep(1)
    print("李四：你好，未来的朋友")
    time.sleep(1)

if __name__ == "__main__":
    for i in range(3):
        hello_for()

张三：你好 世界
张三：hi
李四：你好，未来的朋友
张三：你好 世界
张三：hi
李四：你好，未来的朋友
张三：你好 世界
张三：hi
李四：你好，未来的朋友


In [13]:
"""多线程执行"""
def hello_for():
    print("张三：你好 世界")
    time.sleep(1)
    print("张三：hi")
    time.sleep(1)
    print("李四：你好，未来的朋友")
    time.sleep(1)

if __name__ == "__main__":
    for i in range(3):
        t = threading.Thread(target=hello_for)
        t.start() #启动线程，即让线程开始执行

张三：你好 世界
张三：你好 世界
张三：你好 世界
张三：hi
张三：hi
张三：hi
李四：你好，未来的朋友
李四：你好，未来的朋友
李四：你好，未来的朋友


In [14]:
import threading
from time import sleep,ctime

def sing():
    for i in range(3):
        print("正在唱歌...%d"%i)
        sleep(1)

def dance():
    for i in range(3):
        print("正在跳舞...%d"%i)
        sleep(1)

if __name__ == '__main__':
    print('---开始---:%s'%ctime())

    t1 = threading.Thread(target=sing)
    t2 = threading.Thread(target=dance)

    t1.start()
    t2.start()

    print('---结束---:%s'%ctime())

---开始---:Tue Nov 17 11:20:04 2020
正在唱歌...0
正在跳舞...0
---结束---:Tue Nov 17 11:20:04 2020
正在唱歌...1
正在跳舞...1
正在唱歌...2
正在跳舞...2


In [15]:
import threading
import time
#定义线程需要做的内容，写在函数里面
def target():
    print('当前的线程%s 在运行' % threading.current_thread().name)
    time.sleep(1)
    print('当前的线程 %s 结束' % threading.current_thread().name)

print('当前的线程 %s 在运行' % threading.current_thread().name)
t = threading.Thread(target=target,args = [])
 
t.start()  #线程启动
print('当前的线程 %s 结束' % threading.current_thread().name)


当前的线程 MainThread 在运行
当前的线程Thread-21 在运行
当前的线程 MainThread 结束
当前的线程 Thread-21 结束


In [18]:
import time

def run():

    time.sleep(2)
    print('当前线程的名字是： ', threading.current_thread().name)
    time.sleep(2)


if __name__ == '__main__':

    start_time = time.time()

    print('这是主线程：', threading.current_thread().name)
    thread_list = []
    for i in range(5):
        t = threading.Thread(target=run)
        thread_list.append(t)

    for t in thread_list:
        t.setDaemon(True)
        t.start()

    print('主线程结束了！' , threading.current_thread().name)
    print('一共用时：', time.time()-start_time)


这是主线程： MainThread
主线程结束了！ MainThread
一共用时： 0.0019683837890625
当前线程的名字是： 当前线程的名字是：  Thread-34
当前线程的名字是：  Thread-36
 当前线程的名字是：  Thread-33
当前线程的名字是：  Thread-35
Thread-32


In [3]:
con = threading.Condition()
def job1():
    con.acquire()
    print("JOB1：床前明月光")
    con.notify()   # 唤醒正在等待(wait)的线程
 
    con.wait()   # 等待对方回应消息，使用wait阻塞线程，等待对方通过notify唤醒本线程
    print("JOB1：举头看明月")
    con.notify()  # 唤醒对方
 
    con.release()
 
 
def job2():
    con.acquire()
 
    con.wait()
    print("JOB2:疑似地上桑")
    con.notify()
    con.wait()
    print("JOB2:低头思故乡")
    con.notify()
 
    con.release()
 
 
def main():
    t1 = threading.Thread(target=job1)
    t2 = threading.Thread(target=job2)
 
    t2.start()   #此处注意顺序，先t2,否则双方都处于wait状态，程序卡死
    t1.start()
 
    t1.join()
    t2.join()
 
 
if __name__ == "__main__":
    main()

JOB1：床前明月光
JOB2:疑似地上桑
JOB1：举头看明月
JOB2:低头思故乡
